In [1]:
import pandas as pd

In [ ]:
# read files for merge
df1 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 01.xlsx")
df2 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 02.xlsx")
df3 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 03.xlsx")
df4 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 04.xlsx")
df5 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 05.xlsx")

In [ ]:
df_merge = pd.concat([df1,df2,df3,df4,df5])
df_merge.head()

In [ ]:
df_merge.shape

In [ ]:
df_merge.info()

In [ ]:
# check for missing values and remove
print(df_merge.isnull().sum())
df_merge.dropna(inplace=True)

In [ ]:
# check for duplicate values and remove
print(df_merge.duplicated().sum())
df_merge = df_merge.drop_duplicates(keep='first')

In [ ]:
df_merge.shape